## Youtube video link:
https://youtu.be/WqlMnxlAj5o?si=QilXRBz78v21AsUy

# Custom GPT Link:
https://chatgpt.com/g/g-ptgad0eJV-crewai-to-agency-swarm-converter

# Trip Planner Example
https://github.com/crewAIInc/crewAI-examples/tree/main/trip_planner

## Installation

In [ ]:
%%capture
!pip install git+https://github.com/VRSEN/agency-swarm.git

In [ ]:
%%capture
!pip install crewai==0.11.0 unstructured==0.10.25 pyowm==3.3.0 tools==0.1.9 python-dotenv==1.0.0

In [ ]:
import os
import getpass

# Set environment variables for API keys
os.environ['SERPER_API_KEY'] = getpass.getpass('Enter SERPER_API_KEY: ') # https://serper.dev/ (free tier)
os.environ['BROWSERLESS_API_KEY'] = getpass.getpass('Enter BROWSERLESS_API_KEY: ') # https://www.browserless.io/ (free tier)
os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter OPENAI_API_KEY: ')

Enter SERPER_API_KEY: ··········
Enter BROWSERLESS_API_KEY: ··········
Enter OPENAI_API_KEY: ··········


## Step 1: Convert the Tools


## Browser tools

CrewAI

In [ ]:
import json
import os

import requests
from crewai import Agent, Task
from langchain.tools import tool
from unstructured.partition.html import partition_html


class BrowserTools():
  @tool("Scrape website content")
  def scrape_and_summarize_website(website):
    """Useful to scrape and summarize a website content"""
    url = f"https://chrome.browserless.io/content?token={os.environ['BROWSERLESS_API_KEY']}"
    payload = json.dumps({"url": website})
    headers = {'cache-control': 'no-cache', 'content-type': 'application/json'}
    response = requests.request("POST", url, headers=headers, data=payload)
    elements = partition_html(text=response.text)
    content = "\n\n".join([str(el) for el in elements])
    content = [content[i:i + 8000] for i in range(0, len(content), 8000)]
    summaries = []
    for chunk in content:
      agent = Agent(
          role='Principal Researcher',
          goal=
          'Do amazing researches and summaries based on the content you are working with',
          backstory=
          "You're a Principal Researcher at a big company and you need to do a research about a given topic.",
          allow_delegation=False)
      task = Task(
          agent=agent,
          model="gpt-4o-mini",
          description=
          f'Analyze and summarize the content bellow, make sure to include the most relevant information in the summary, return only the summary nothing else.\n\nCONTENT\n----------\n{chunk}'
      )
      summary = task.execute()
      summaries.append(summary)
    return "\n\n".join(summaries)

Agency Swarm

In [ ]:
import json
import os
import requests
from agency_swarm.tools import BaseTool
from pydantic import Field
from unstructured.partition.html import partition_html
from agency_swarm import get_openai_client

class ScrapeAndSummarizeWebsite(BaseTool):
    """
    Tool for scraping and summarizing website content.
    """
    website: str = Field(..., description="The URL of the website to scrape and summarize.")

    def run(self):
        url = f"https://chrome.browserless.io/content?token={os.environ['BROWSERLESS_API_KEY']}"
        payload = json.dumps({"url": self.website})
        headers = {'cache-control': 'no-cache', 'content-type': 'application/json'}
        response = requests.request("POST", url, headers=headers, data=payload)
        elements = partition_html(text=response.text)
        content = "\n\n".join([str(el) for el in elements])
        content = [content[i:i + 8000] for i in range(0, len(content), 8000)]
        summaries = []
        client = get_openai_client()

        for chunk in content:
            task_description = dedent(f"""
                Analyze and summarize the content below, make sure to include the most relevant information in the summary, return only the summary nothing else.

                CONTENT
                ----------
                {chunk}
            """)
            completion = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are a Principal Researcher at a big company and you need to do research about a given topic."},
                    {"role": "user", "content": task_description}
                ],
            )
            summary = completion.choices[0].message.content
            summaries.append(summary)

        return "\n\n".join(summaries)


## Calculator Tools

CrewAI

In [ ]:
from langchain.tools import tool

class CalculatorTools():
    @tool("Make a calculation")
    def calculate(operation):
        """Useful to perform any mathematical calculations,
        like sum, minus, multiplication, division, etc.
        The input to this tool should be a mathematical
        expression, a couple examples are `200*7` or `5000/2*10`
        """
        try:
            return eval(operation)
        except SyntaxError:
            return "Error: Invalid syntax in mathematical expression"

Agency Swarm

In [ ]:
from agency_swarm.tools import BaseTool
from pydantic import Field

class CalculatorTool(BaseTool):
    """
    Tool for performing mathematical calculations such as sum, minus, multiplication, division, etc.
    """
    operation: str = Field(..., description="A mathematical expression to be evaluated, e.g., '200*7' or '5000/2*10'.")

    def run(self):
        try:
            result = eval(self.operation)
            return str(result)
        except SyntaxError:
            return "Error: Invalid syntax in mathematical expression"


## Search Tools

CrewAI

In [ ]:
import json
import os

import requests
from langchain.tools import tool


class SearchTools():

  @tool("Search the internet")
  def search_internet(query):
    """Useful to search the internet
    about a a given topic and return relevant results"""
    top_result_to_return = 4
    url = "https://google.serper.dev/search"
    payload = json.dumps({"q": query})
    headers = {
        'X-API-KEY': os.environ['SERPER_API_KEY'],
        'content-type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    # check if there is an organic key
    if 'organic' not in response.json():
      return "Sorry, I couldn't find anything about that, there could be an error with you serper api key."
    else:
      results = response.json()['organic']
      string = []
      for result in results[:top_result_to_return]:
        try:
          string.append('\n'.join([
              f"Title: {result['title']}", f"Link: {result['link']}",
              f"Snippet: {result['snippet']}", "\n-----------------"
          ]))
        except KeyError:
          next

      return '\n'.join(string)

Agency Swarm

In [ ]:
import json
import os
import requests
from agency_swarm.tools import BaseTool
from pydantic import Field

class SearchInternet(BaseTool):
    """
    Tool for searching the internet about a given topic and returning relevant results.
    """
    query: str = Field(..., description="The search query for the internet search.")

    def run(self):
        top_result_to_return = 4
        url = "https://google.serper.dev/search"
        payload = json.dumps({"q": self.query})
        headers = {
            'X-API-KEY': os.environ['SERPER_API_KEY'],
            'content-type': 'application/json'
        }
        response = requests.request("POST", url, headers=headers, data=payload)
        if 'organic' not in response.json():
            return "Sorry, I couldn't find anything about that, there could be an error with your Serper API key."
        else:
            results = response.json()['organic']
            string = []
            for result in results[:top_result_to_return]:
                try:
                    string.append('\n'.join([
                        f"Title: {result['title']}",
                        f"Link: {result['link']}",
                        f"Snippet: {result['snippet']}",
                        "\n-----------------"
                    ]))
                except KeyError:
                    continue

            return '\n'.join(string)

# Step 2: Convert the Agents

Use the following prompt: "Convert this into agency swarm agents, using the existing API parameters:"

In [ ]:
from crewai import Agent
from langchain.llms import OpenAI

class TripAgentsCrew():
  def city_selection_agent(self):
    return Agent(
        role='City Selection Expert',
        goal='Select the best city based on weather, season, and prices',
        backstory=
        'An expert in analyzing travel data to pick ideal destinations',
        tools=[
            SearchTools.search_internet,
            BrowserTools.scrape_and_summarize_website,
        ],
        model="gpt-4o-mini",
        verbose=True)

  def local_expert(self):
    return Agent(
        role='Local Expert at this city',
        goal='Provide the BEST insights about the selected city',
        backstory="""A knowledgeable local guide with extensive information
        about the city, it's attractions and customs""",
        tools=[
            SearchTools.search_internet,
            BrowserTools.scrape_and_summarize_website,
        ],
        model="gpt-4o-mini",
        verbose=True)

  def travel_concierge(self):
    return Agent(
        role='Amazing Travel Concierge',
        goal="""Create the most amazing travel itineraries with budget and
        packing suggestions for the city""",
        backstory="""Specialist in travel planning and logistics with
        decades of experience""",
        tools=[
            SearchTools.search_internet,
            BrowserTools.scrape_and_summarize_website,
            CalculatorTools.calculate,
        ],
        model="gpt-4o-mini",
        verbose=True)

In [ ]:
from agency_swarm import Agent

class TripAgentsAgency:
    def city_selection_agent(self):
        return Agent(
            name="City Selection Expert",
            description="An expert in analyzing travel data to pick ideal destinations",
            instructions="Select the best city based on weather, season, and prices.",
            tools=[
                SearchInternet,
                ScrapeAndSummarizeWebsite,
            ],
            model="gpt-4o-mini",
        )

    def local_expert(self):
        return Agent(
            name="Local Expert at this city",
            description="A knowledgeable local guide with extensive information about the city, its attractions, and customs.",
            instructions="Provide the BEST insights about the selected city.",
            tools=[
                SearchInternet,
                ScrapeAndSummarizeWebsite,
            ],
            model="gpt-4o-mini",
        )

    def travel_concierge(self):
        return Agent(
            name="Amazing Travel Concierge",
            description="Specialist in travel planning and logistics with decades of experience.",
            instructions="Create the most amazing travel itineraries with budget and packing suggestions for the city.",
            tools=[
                SearchInternet,
                ScrapeAndSummarizeWebsite,
                CalculatorTool,
            ],
            model="gpt-4o-mini",
        )


# Step 3: Convert Tasks

Use the following prompt: "Convert these tasks into agency.get_completion methods. Pass the agent into recepient_agent parameter:"

In [ ]:
from crewai import Task
from textwrap import dedent
from datetime import date


class TripTasksCrew():

  def identify_task(self, agent, origin, cities, interests, range):
    return Task(description=dedent(f"""
        Analyze and select the best city for the trip based
        on specific criteria such as weather patterns, seasonal
        events, and travel costs. This task involves comparing
        multiple cities, considering factors like current weather
        conditions, upcoming cultural or seasonal events, and
        overall travel expenses.

        Your final answer must be a detailed
        report on the chosen city, and everything you found out
        about it, including the actual flight costs, weather
        forecast and attractions.
        {self.__tip_section()}

        Traveling from: {origin}
        City Options: {cities}
        Trip Date: {range}
        Traveler Interests: {interests}
      """),
                agent=agent)

  def gather_task(self, agent, origin, interests, range):
    return Task(description=dedent(f"""
        As a local expert on this city you must compile an
        in-depth guide for someone traveling there and wanting
        to have THE BEST trip ever!
        Gather information about  key attractions, local customs,
        special events, and daily activity recommendations.
        Find the best spots to go to, the kind of place only a
        local would know.
        This guide should provide a thorough overview of what
        the city has to offer, including hidden gems, cultural
        hotspots, must-visit landmarks, weather forecasts, and
        high level costs.

        The final answer must be a comprehensive city guide,
        rich in cultural insights and practical tips,
        tailored to enhance the travel experience.
        {self.__tip_section()}

        Trip Date: {range}
        Traveling from: {origin}
        Traveler Interests: {interests}
      """),
                agent=agent)

  def plan_task(self, agent, origin, interests, range):
    return Task(description=dedent(f"""
        Expand this guide into a a full 7-day travel
        itinerary with detailed per-day plans, including
        weather forecasts, places to eat, packing suggestions,
        and a budget breakdown.

        You MUST suggest actual places to visit, actual hotels
        to stay and actual restaurants to go to.

        This itinerary should cover all aspects of the trip,
        from arrival to departure, integrating the city guide
        information with practical travel logistics.

        Your final answer MUST be a complete expanded travel plan,
        formatted as markdown, encompassing a daily schedule,
        anticipated weather conditions, recommended clothing and
        items to pack, and a detailed budget, ensuring THE BEST
        TRIP EVER, Be specific and give it a reason why you picked
        # up each place, what make them special! {self.__tip_section()}

        Trip Date: {range}
        Traveling from: {origin}
        Traveler Interests: {interests}
      """),
                agent=agent)

  def __tip_section(self):
    return "If you do your BEST WORK, I'll tip you $100!"

In [ ]:
from textwrap import dedent
from datetime import date

class TripTasksAgency:

    def identify_task(self, agency, agent, origin, cities, interests, date_range):
        task_description = dedent(f"""
            Analyze and select the best city for the trip based
            on specific criteria such as weather patterns, seasonal
            events, and travel costs. This task involves comparing
            multiple cities, considering factors like current weather
            conditions, upcoming cultural or seasonal events, and
            overall travel expenses.

            Your final answer must be a detailed
            report on the chosen city, and everything you found out
            about it, including the actual flight costs, weather
            forecast and attractions.
            {self.__tip_section()}

            Traveling from: {origin}
            City Options: {cities}
            Trip Date: {date_range}
            Traveler Interests: {interests}
        """)

        return agency.get_completion(
            message=task_description,
            recipient_agent=agent,
            verbose=True
        )

    def gather_task(self, agency, agent, origin, interests, date_range):
        task_description = dedent(f"""
            As a local expert on this city you must compile an
            in-depth guide for someone traveling there and wanting
            to have THE BEST trip ever!
            Gather information about key attractions, local customs,
            special events, and daily activity recommendations.
            Find the best spots to go to, the kind of place only a
            local would know.
            This guide should provide a thorough overview of what
            the city has to offer, including hidden gems, cultural
            hotspots, must-visit landmarks, weather forecasts, and
            high level costs.

            The final answer must be a comprehensive city guide,
            rich in cultural insights and practical tips,
            tailored to enhance the travel experience.
            {self.__tip_section()}

            Trip Date: {date_range}
            Traveling from: {origin}
            Traveler Interests: {interests}
        """)

        return agency.get_completion(
            message=task_description,
            recipient_agent=agent,
            verbose=True
        )

    def plan_task(self, agency, agent, origin, interests, date_range):
        task_description = dedent(f"""
            Expand this guide into a full 7-day travel
            itinerary with detailed per-day plans, including
            weather forecasts, places to eat, packing suggestions,
            and a budget breakdown.

            You MUST suggest actual places to visit, actual hotels
            to stay and actual restaurants to go to.

            This itinerary should cover all aspects of the trip,
            from arrival to departure, integrating the city guide
            information with practical travel logistics.

            Your final answer MUST be a complete expanded travel plan,
            formatted as markdown, encompassing a daily schedule,
            anticipated weather conditions, recommended clothing and
            items to pack, and a detailed budget, ensuring THE BEST
            TRIP EVER. Be specific and give it a reason why you picked
            each place, what makes them special! {self.__tip_section()}

            Trip Date: {date_range}
            Traveling from: {origin}
            Traveler Interests: {interests}
        """)

        return agency.get_completion(
            message=task_description,
            recipient_agent=agent,
            verbose=True
        )

    def __tip_section(self):
        return "If you do your BEST WORK, I'll tip you $100!"

# Step 4: Convert Crew

Use the following prompt: "Convert this into Agency:"

In [ ]:
%%time

from crewai import Crew, Agent
from textwrap import dedent

from dotenv import load_dotenv
load_dotenv()

class TripCrew:

  def __init__(self, origin, cities, date_range, interests):
    self.cities = cities
    self.origin = origin
    self.interests = interests
    self.date_range = date_range

  def run(self):
    agents = TripAgentsCrew()
    tasks = TripTasksCrew()

    city_selector_agent = agents.city_selection_agent()
    local_expert_agent = agents.local_expert()
    travel_concierge_agent = agents.travel_concierge()

    identify_task = tasks.identify_task(
      city_selector_agent,
      self.origin,
      self.cities,
      self.interests,
      self.date_range
    )
    gather_task = tasks.gather_task(
      local_expert_agent,
      self.origin,
      self.interests,
      self.date_range
    )
    plan_task = tasks.plan_task(
      travel_concierge_agent,
      self.origin,
      self.interests,
      self.date_range
    )

    crew = Crew(
      agents=[
        city_selector_agent, local_expert_agent, travel_concierge_agent
      ],
      tasks=[identify_task, gather_task, plan_task],
      verbose=True
    )

    result = crew.kickoff()
    return result

print("## Welcome to Trip Planner Crew")
print('-------------------------------')
location = input(
  dedent("""
    From where will you be traveling from?
  """))
cities = input(
  dedent("""
    What are the cities options you are interested in visiting?
  """))
date_range = input(
  dedent("""
    What is the date range you are interested in traveling?
  """))
interests = input(
  dedent("""
    What are some of your high level interests and hobbies?
  """))

trip_crew = TripCrew(location, cities, date_range, interests)
crew_result = trip_crew.run()
print("\n\n########################")
print("## Here is you Trip Plan")
print("########################\n")
print(crew_result)

## Welcome to Trip Planner Crew
-------------------------------

From where will you be traveling from?
London

What are the cities options you are interested in visiting?
Madrid

What is the date range you are interested in traveling?
1 week

What are some of your high level interests and hobbies?
Hiking


[DEBUG]: Working Agent: City Selection Expert
[INFO]: Starting Task: 
Analyze and select the best city for the trip based
on specific criteria such as weather patterns, seasonal
events, and travel costs. This task involves comparing
multiple cities, considering factors like current weather
conditions, upcoming cultural or seasonal events, and
overall travel expenses.

Your final answer must be a detailed
report on the chosen city, and everything you found out
about it, including the actual flight costs, weather
forecast and attractions.
If you do your BEST WORK, I'll tip you $100!

Traveling from: London
City Options: Madrid
Trip Date: 1 week
Traveler Interests: Hiking



> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Search the internet
Action Input: Search for the current weather forecast in Madrid for the next week, any upcoming events, and the average travel costs from London to Madrid. 

Title: 10-Day Weather Forecast for Valdeacederas, Mad

/usr/local/lib/python3.10/dist-packages/unstructured/documents/html.py:498: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  rows = body.findall("tr") if body else []


 

The weather in Madrid, Spain, over the next few weeks is forecasted to be generally hot, with temperatures ranging from the mid-60s to over 100°F. The highest temperatures are predicted for July 23rd - 26th, and July 31st - August 1st, all exceeding 100°F. The weather will be predominantly sunny, with some days experiencing passing or broken clouds. The lowest temperature recorded is 65°F on July 22nd and the highest is 108°F on July 31st. No significant precipitation is mentioned in the forecast.

Do I need to use a tool? Yes
Action: Search the internet
Action Input: Average cost of living in Madrid 

Title: Cost of Living in Madrid
Link: https://www.numbeo.com/cost-of-living/in/Madrid
Snippet: A single person estimated monthly costs are 887.5$ (815.1€) without rent. Madrid is 46.6% less expensive than New York (without rent). Rent in Madrid is ...

-----------------
Title: The ultimate guide to the cost of living in Madrid - HousingAnywhere
Link: https://housinganywhere.com/Madrid

/usr/local/lib/python3.10/dist-packages/unstructured/documents/html.py:498: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  rows = body.findall("tr") if body else []


 

The article provides information about the cost of living in Madrid, Spain. It identifies the city as the 10th best globally, with warm and sunny weather, excellent public transport, high-quality education, and endless job opportunities. The cost of living is divided into four categories: housing, grocery, leisure activities, and transportation. 

Average monthly rents for different types of housing are provided, with a private room costing €550, a studio €995, and an apartment €1,494. The article warns about rental scams due to high demand, especially at the end of summer and beginning of autumn, and recommends starting the housing search as early as May.

Grocery costs vary, with discount stores and supermarket chains offering options for budget-conscious residents. Average prices for common food items are provided. Leisure activities and their costs are also discussed, covering everything from museum tickets to dining out. 

In terms of transportation, Madrid offers a good public

In [ ]:
%%time
from textwrap import dedent
from dotenv import load_dotenv
from agency_swarm import Agency, Agent

load_dotenv()

class TripAgency:

    def __init__(self, origin, cities, date_range, interests):
        self.cities = cities
        self.origin = origin
        self.interests = interests
        self.date_range = date_range

    def run(self):
        agents = TripAgentsAgency()
        tasks = TripTasksAgency()

        city_selector_agent = agents.city_selection_agent()
        local_expert_agent = agents.local_expert()
        travel_concierge_agent = agents.travel_concierge()

agency = Agency(
    agency_chart=[
        city_selector_agent, local_expert_agent, travel_concierge_agent,
    ]
)

        identify_task = tasks.identify_task(
            agency,
            city_selector_agent,
            self.origin,
            self.cities,
            self.interests,
            self.date_range
        )
        gather_task = tasks.gather_task(
            agency,
            local_expert_agent,
            self.origin,
            self.interests,
            self.date_range
        )
        plan_task = tasks.plan_task(
            agency,
            travel_concierge_agent,
            self.origin,
            self.interests,
            self.date_range
        )

        tasks_list = [identify_task, gather_task, plan_task]

        # Execute tasks in sequence
        results = []
        for task in tasks_list:
            result = task
            results.append(result)

        return plan_task


print("## Welcome to Trip Planner Agency")
print('-------------------------------')
location = input(
    dedent("""
        From where will you be traveling from?
    """))
cities = input(
    dedent("""
        What are the cities options you are interested in visiting?
    """))
date_range = input(
    dedent("""
        What is the date range you are interested in traveling?
    """))
interests = input(
    dedent("""
        What are some of your high level interests and hobbies?
    """))

trip_agency = TripAgency(location, cities, date_range, interests)
# Assuming `agency` is initialized elsewhere and passed here
agency_result = trip_agency.run()
print("\n\n########################")
print("## Here is your Trip Plan")
print("########################\n")
print(agency_result)

## Welcome to Trip Planner Agency
-------------------------------

From where will you be traveling from?
London

What are the cities options you are interested in visiting?
Madrid

What is the date range you are interested in traveling?
1 week

What are some of your high level interests and hobbies?
Hiking
Updating agent... City Selection Expert
Updating agent... Local Expert at this city
Updating agent... Amazing Travel Concierge
THREAD:[ user -> City Selection Expert ]: URL https://platform.openai.com/playground/assistants?assistant=asst_tLpgHBIqNo6EkPmWYgK2sZd8&mode=assistant&thread=thread_wRnXh17sygqmInRmayoKgjms


───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

👤 User 🗣️ @City Selection Expert
Analyze and select the best city for the trip based on specific criteria such as weather patterns, seasonal events,
and travel costs. This task involves comparing multiple cities, considering factors like current weather           
conditions, upcoming cultural or seasonal events, and overall travel expenses.                                     

Your final answer must be a detailed report on the chosen city, and everything you found out about it, including   
the actual flight costs, weather forecast and attractions. If you do your BEST WORK, I'll tip you $100!            

Traveling from: London City Options: Madrid Trip Date: 1 week Traveler Interests: Hiking

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

🐤 City Selection Expert 🛠️ Executing Function
Function(arguments='{"query": "current weather Madrid October 2023"}', name='SearchInternet')

...

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

ScrapeAndSummarizeWebsite ⚙️ Function Output
The article reviews the top 25 restaurants in Madrid, highlighting their diverse culinary offerings. Key mentions  
include:                                                                                                           

  1 DiverXO - Known for its innovative Michelin-starred cuisine by Dabiz Muñoz, featuring a long waiting list and a
    memorable, elaborate dining experience.                                                                        
  2 Montia - Recently reopened by chef Dani Ochoa with a focus on local ingredients and a Michelin star in 2023.   
  3 VelascoAbellà - A new venture by Óscar Velasco and Montse Abellà, noted for its intimate setting and exquisite 
    dishes.                                                                                                        
  4 Ugo Chan - Renowned for its Japanese Omakase tasting menu and minimalistic yet elegant presentation.           
  5 Desde 1911 - Offers a seasonal, high-cost menu centered on fresh catch-of-the-day seafood.                     
  6 Deessa - A Michelin two-star restaurant known for its detailed and elegant dishes served with impeccable       
    service.                                                                                                       
  7 Smoked Room - Unique for its smoke-focused haute cuisine and interactive dining experience; awarded two        
    Michelin stars simultaneously.                                                                                 
  8 Corral de la Morería - A flamenco tablao with a Michelin star restaurant; focuses on Basque cuisine and an     
    extensive Jerez wine selection.                                                                                
  9 El Invernadero - Specializes in high-end vegetable-centric cuisine with unexpected pairings and Michelin-star  
    quality.                                                                                                       
 10 Coque - A family-run, two Michelin-starred restaurant known for its innovative use of a cutting-edge oven to   
    create distinctive dishes.                                                                                     

These restaurants showcase the richness and diversity of Madrid's culinary landscape, from traditional local       
flavors to innovative international dishes.                                                                        

The content provides detailed descriptions of some of the best restaurants in Madrid, emphasizing each             
establishment's unique offerings and culinary styles. Notable mentions include:                                    

 • La Tasquita de Enfrente: Emphasizes top-quality ingredients and transforms traditional recipes into             
   contemporary, elegant dishes.                                                                                   
 • DSTAgE: Known for its haute cuisine with a global influence, offering two tasting menus.                        
 • Estimar Madrid: Focuses on Mediterranean cuisine, advocating simplicity with limited ingredients to highlight   
   product quality.                                                                                                
 • Lana: An Argentinean restaurant with a focus on quality meats and a fine wine selection.                        
 • La Tasquería: Specializes in offal, transforming it into contemporary dishes with various animal parts.         
 • Cebo: Combines Mediterranean and local Madrid cuisine, featuring elaborate tasting menus in a prestigious hotel.
 • Trèsde: Offers a seasonally changing menu with a unique twist on traditional dining.                            
 • Sacha: Blends French influences with seasonal and classic creations in a distinctive setting.                   
 • Horcher: A historic restaurant known for its timeless elegance and unique dishes.                               
 • Laka

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

🐸 Amazing Travel Concierge 🗣️ @User
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                        7-Day Travel Itinerary for Madrid                                        ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Welcome to your ultimate 7-day travel itinerary for Madrid! This plan is tailored for a week-long adventure filled 
with hiking, cultural exploration, and culinary delights. Below, you'll find detailed daily schedules, weather     
forecasts, packing suggestions, and a budget breakdown to ensure you have the best trip ever.                      

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

                                             Day 1: Arrival in Madrid                                              

                                                 Weather Forecast                                                  

 • Temperature: 67°F (19°C), sunny                                                                                 
 • Clothing: Light layers, comfortable shoes for walking                                                           

                                                      Morning                                                      

 • Flight from London to Madrid: Arrive at Madrid-Barajas Airport.                                                 
 • Transportation: Take Metro Line 8 to Nuevos Ministerios, then transfer to Line 10 to reach your hotel.          

                                                   Accommodation                                                   

 • Hotel: The Hat Madrid                                                                                           
   Budget-friendly, eco-conscious hotel with a rooftop terrace.                                                    
   Cost: €80 per night                                                                                             

                                                     Afternoon                                                     

 • Lunch: Mercado de San Miguel                                                                                    
   Enjoy a variety of tapas and local delicacies.                                                                  
   Cost: €15                                                                                                       

                                                      Evening                                                      

 • Activity: Stroll through Plaza Mayor and Puerta del Sol.                                                        
 • Dinner: Casa Lucio                                                                                              
   Famous for its traditional Spanish dishes, especially the huevos rotos.                                         
   Cost: €25                                                                                                       

                                                 Budget Breakdown                                                  

 • Accommodation: €80                                                                                              
 • Meals: €40                                                                                                      
 • Transportation: €5                                                                                              
 • Total: €125                                                                                                     

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

                                              Day 2: Art and Culture                                               

  



########################
## Here is your Trip Plan
########################

# 7-Day Travel Itinerary for Madrid

Welcome to your ultimate 7-day travel itinerary for Madrid! This plan is tailored for a week-long adventure filled with hiking, cultural exploration, and culinary delights. Below, you'll find detailed daily schedules, weather forecasts, packing suggestions, and a budget breakdown to ensure you have the best trip ever.

---

## **Day 1: Arrival in Madrid**

### **Weather Forecast**
- **Temperature**: 67°F (19°C), sunny
- **Clothing**: Light layers, comfortable shoes for walking

### **Morning**
- **Flight from London to Madrid**: Arrive at Madrid-Barajas Airport.
- **Transportation**: Take Metro Line 8 to Nuevos Ministerios, then transfer to Line 10 to reach your hotel.

### **Accommodation**
- **Hotel**: **The Hat Madrid**  
  *Budget-friendly, eco-conscious hotel with a rooftop terrace.*  
  **Cost**: €80 per night

### **Afternoon**
- **Lunch**: **Mercado de San Miguel*

# Compare the results

In [ ]:
from agency_swarm.messages import MessageOutput

In [ ]:
#@title Crew AI
MessageOutput("text", content=crew_result, sender_name="CrewAI Results", receiver_name="User").cprint()

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

🐮 CrewAI Results 🗣️ @User
Here's your detailed 7-day travel itinerary for Madrid, complete with weather forecasts, places to eat, packing    
suggestions, and a budget breakdown:                                                                               

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                      Day 1: Arrival and Exploring the City                                      ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

 • Arrival at Madrid-Barajas Airport and transfer to your hotel, the Four Seasons Hotel Madrid, offering a blend of
   historic architecture and modern comfort.                                                                       
 • After settling in, begin your exploration with Gran Vía, the city's most famous shopping street. It's the       
   perfect place to immerse yourself in the local atmosphere.                                                      
 • Dinner at Sacha, a restaurant known for its comfort food.                                                       
 • Budget spend: Flight cost, Hotel (approx. $250 per night), Meals (approx. $80), Local Transportation (approx.   
   $30)                                                                                                            

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                             Day 2: Art and History                                              ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

 • Visit the Museo del Prado, Spain's most important art museum.                                                   
 • Lunch at Casa Dani, renowned for its comforting potato omelet.                                                  
 • Explore the Royal Palace, the former residence of the Spanish monarchy.                                         
 • Dinner at DiverXO, a three-Michelin-star restaurant known for its avant-garde cooking.                          
 • Budget spend: Museum and Palace Entrance Fees (approx. $30), Meals (approx. $120), Local Transportation (approx.
   $30)                                                                                                            

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                Day 3: Exploring Neighbourhoods and Local Cuisine                                ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

 • Explore the neighborhoods of Malasaña and Chueca, known for their vibrant bar and café scene.                   
 • Lunch at La Gildería, a family-owned restaurant known for its Spanish tortillas.                                
 • Visit Mercado de San Ildefonso and try Madrid's traditional dishes like tapas and paella.                       
 • Dinner at Desde 1911, specializing in seafood.                                                                  
 • Budget spend: Meals (approx. $100), Local Transportation (approx. $30)                                          

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                      Day 4: Outdoor Activities and Hiking                                       ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

 • Take a hike on Siete Picos, a family-friendly trail offering panoramic views.                                   
 • Lunch at Toga, known for fusion dishes like noodles with kimchi mayo and raw red tuna.                          
 • In the evening, rel

In [ ]:
#@title Agency Swarm
MessageOutput("text", content=agency_result, sender_name="CrewAI Results", receiver_name="User").cprint()

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

🐮 CrewAI Results 🗣️ @User
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                        7-Day Travel Itinerary for Madrid                                        ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛


                                                     Overview                                                      

This comprehensive itinerary is designed for a week-long trip to Madrid, focusing on hiking, cultural experiences, 
and local cuisine. Each day includes weather forecasts, recommended activities, dining options, and budget         
estimates.                                                                                                         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

                                             Day 1: Arrival in Madrid                                              

                                                 Weather Forecast                                                  

 • Temperature: 20°C (68°F), partly cloudy                                                                         
 • Packing Suggestions: Light jacket, comfortable walking shoes, and a small backpack.                             

                                                    Activities                                                     

 • Arrival at Madrid-Barajas Airport: Take the Metro Line 8 to Nuevos Ministerios and transfer to Line 10 to reach 
   your hotel.                                                                                                     
 • Check-in at Hotel:                                                                                              
    • Hotel Recommendation: The Principal Madrid (4-star) - Centrally located, stylish, and offers great amenities.
    • Cost: Approximately €150 per night.                                                                          

                                                      Evening                                                      

 • Dinner at: Casa Lucio - Famous for its huevos rotos (broken eggs) and traditional Spanish cuisine.              
 • Budget: €30 per person.                                                                                         

                                                 Total Day Budget                                                  

 • Accommodation: €150                                                                                             
 • Meals: €30                                                                                                      
 • Transport: €10 (Metro)                                                                                          
 • Total: €190                                                                                                     

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

                                          Day 2: Exploring Central Madrid                                          

                                                 Weather Forecast                                                  

 • Temperature: 22°C (72°F), sunny                                                                                 
 • Packing Suggestions: Light clothing, sunscreen, and a hat.                                                      

                                                    Activities                                                     

 • Morning: Visit Puerta del Sol and Plaza Mayor.                                                                  
 • Lunch at: La Tasquería - A unique dining experience focusing on offal dishes.                                   
 • Afternoo

In [ ]:
from agency_swarm import get_openai_client

client = get_openai_client()

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Compare which travel plan is better. First provide your reasoning, then evaluation. Only output 1 option"},
        {"role": "user", "content": "CrewAI Travel Plan:" + crew_result + "\n\nAgency Swarm Travel Plan:" + agency_result},
    ],
    temperature=0,
)

print(completion.choices[0].message.content)

### Reasoning

**CrewAI Travel Plan:**
1. **Accommodation:** Four Seasons Hotel Madrid, a luxury option at $250 per night.
2. **Dining:** High-end dining options like DiverXO and Sacha, with a focus on gourmet experiences.
3. **Activities:** A mix of cultural sites (Museo del Prado, Royal Palace), local markets (El Rastro), and outdoor activities (Siete Picos hike).
4. **Budget:** Approximately $2380 + flight costs + market purchases.
5. **Weather:** Warm, with temperatures ranging from 65°F to 108°F.

**Agency Swarm Travel Plan:**
1. **Accommodation:** The Principal Madrid, a 4-star hotel at €150 per night.
2. **Dining:** Traditional and local dining options like Casa Lucio and La Tasquería.
3. **Activities:** A balanced mix of cultural sites (Royal Palace, Museo del Prado), local markets (El Rastro), and multiple hiking trips (Sierra de Guadarrama, Camino de los Campamentos).
4. **Budget:** Approximately €1460.
5. **Weather:** Mild, with temperatures ranging from 18°C to 22°C (64°F t